In [ ]:
colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    root = '/content/drive/MyDrive/colab/sat-images/'
    device = 'cuda'
    !wget -O sat.zip https://github.com/lacykaltgr/satellite-image-segmentation/archive/refs/heads/laci420.zip
    !unzip sat.zip
    !cp -r satellite-image-segmentation-laci420/src .
else:
    root = '/Users/laszlofreund/code/ai/satellite-image-segmentation/data/'
    device = 'mps'
    import sys
    import os
    # Add the parent directory to the Python path
    parent_directory = os.path.abspath(os.path.join(os.getcwd(), '..'))
    sys.path.append(parent_directory)

In [ ]:
import torch
from torch.utils.data import DataLoader
from src.dataset import XYDataset

dataset = XYDataset(root='data/')
train_dataset = dataset[:int(0.9*len(dataset))]
val_dataset = dataset[int(0.9*len(dataset)):]

batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [1]:
import torch
from torch import nn
from unet import UNet
from utils import EarlyStopper
from src.utils import binary_to_rgb

config = dict(
    in_channels=3,
    out_channels=1,
    hiddens=[64, 128, 256],
    dropouts=[0, 0.3, 0.3],  # hiddens
    maxpools=2,  # hiddens - 1
    kernel_sizes=3,  # 2*hiddens + 3*hiddens + 2
    paddings=[[(1, 1), 'same'] + ['same']*(2*2), ['same']*int(3*3+2)],  # 2*hiddens + 3*hiddens + 2
    strides=[[2] + [1] * int(3*2-1), [1] * int(3*3+2)],  # 2*hiddens + 3*hiddens
    criterion=nn.BCELoss,
    output_activation=nn.Softmax,
    activation=nn.ReLU,
    post_process=binary_to_rgb,
    dimensions=2,
    device='cuda'
)
unet = UNet(**config)

In [2]:
unet.summary(input_shape=(1, 3, 4000, 3000))

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Conv2d: 2, ReLU: 1, Conv2d: 2, ReLU: 1, BatchNorm2d: 2, Dropout2d: 2, MaxPool2d: 2, Conv2d: 2, ReLU: 1, Conv2d: 2, ReLU: 1, BatchNorm2d: 2, Dropout2d: 2, MaxPool2d: 2, Conv2d: 2, ReLU: 1, Conv2d: 2, ReLU: 1, BatchNorm2d: 2, Dropout2d: 2, Conv2d: 2, ReLU: 1, Conv2d: 2, ReLU: 1, Conv2d: 2, ReLU: 1, BatchNorm2d: 2, BatchNorm2d: 2, Conv2d: 2, ReLU: 1, Conv2d: 2, ReLU: 1, Conv2d: 2, ReLU: 1]

In [ ]:
es = EarlyStopper(patience=10)
unet.train_model(train_loader, val_loader, es, num_epochs=1000, learning_rate=0.0001, device=device)

In [ ]:
predictions, results = unet.predict(val_loader, device=device)

In [ ]:
from matplotlib.pyplot import imshow, show
import numpy as np
imshow(x_train[0])
show()

In [ ]:
from src.utils import conf_matrix
conf_matrix(val_dataset, results)

In [ ]:
unet.load_state_dict(torch.load('/Users/laszlofreund/code/ai/satellite-image-segmentation/unet.h5', map_location=device))

In [ ]:
eval = unet.evaluate(val_loader, device=device)